In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dlc_bci as bci
from utility import * 

## Get Data
Fetches the data from the server and saves it to a folder at the root 

In [3]:
train_input_100, train_target_100, test_input_100, test_target_100 = import100HzData()
train_input, train_target, test_input, test_target = import1000HzData()

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])
<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
print(train_input.shape) 
print(train_target.shape) 
print(test_input.shape) 
print(test_target.shape) 

torch.Size([316, 28, 500])
torch.Size([316])
torch.Size([100, 28, 500])
torch.Size([100])


## Visualize the data

In [5]:
#rawDataForSingleElectrodeVisualization(train_input)

# Neural Net - CNN

In [6]:
# Copy the neural network from the Neural Networks section before and modify it to take 18-channel signal (instead of 1-channel images as it was defined).

2. Define a Convolution Neural Network

In [7]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.n_filters_time=25
        self.n_filters_spat=25
        self.n_filters_conv=25
        self.n_input_channel=28
        self.final_conv_length=2 #!!!!!
        self.filter_time_length=10
        self.conv_stride = 1 #Stride after pooling 
        self.pool_stride = 3 #Stride after pooling
        self.batch_norm_alpha=0.1
        self.pool_time_length=3
        self.pool_time_stride=3
        self.n_filters_2=50
        self.filter_length_2=10
        self.n_filters_3=100
        self.filter_length_3=10
        self.n_filters_4=200
        self.filter_length_4=10
        self.drop_prob=0.3
        self.batch_norm=True
        self.n_classes=2

        # Layer 1 - REMINDER nn.Conv2d(nbChannels, outputChannels, kxk Square convolution)
        self.a = nn.Conv2d(1, self.n_filters_time, (1, self.filter_time_length), stride = 1) #'Time Convolution' 
        self.b = nn.Conv2d(25, self.n_filters_spat, (self.n_input_channel, 1), stride=(self.conv_stride, 1), bias=False) #'Spatial Convolution' 
        self.c = nn.BatchNorm2d(self.n_filters_conv, momentum=self.batch_norm_alpha, affine=True, eps=1e-5)
        self.d = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))

        # Layer 2
        self.e = nn.Dropout(p=self.drop_prob)
        self.f = nn.Conv2d(self.n_filters_conv, self.n_filters_2,(1, self.filter_length_2),stride=(1, self.conv_stride),
                                   bias=False)
        self.g = nn.BatchNorm2d(self.n_filters_2,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.h = nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride))

        # Layer 3
        self.i = nn.Dropout(p=self.drop_prob)
        self.j = nn.Conv2d(self.n_filters_2, self.n_filters_3, (1, self.filter_length_3), stride=(1, self.conv_stride),
                                   bias=False)
        self.k = nn.BatchNorm2d(self.n_filters_3,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.l = nn.MaxPool2d(kernel_size=(1, self.pool_time_length), stride=(1, self.pool_stride))

        # Layer 4
        self.m =nn.Dropout(p=self.drop_prob)
        self.n =nn.Conv2d(self.n_filters_3, self.n_filters_4,
                                   (1, self.filter_length_4),
                                   stride=(1, self.conv_stride),
                                   bias=False)
        self.o =nn.BatchNorm2d(self.n_filters_4,momentum=self.batch_norm_alpha,affine=True,eps=1e-5)
        self.p =nn.MaxPool2d(kernel_size=(1, self.pool_time_length),stride=(1, self.pool_stride-1))

        #Classifier Layer
        self.q = nn.Conv2d(self.n_filters_4, self.n_classes, (1, self.final_conv_length), bias=True)
        self.r =nn.LogSoftmax(dim=-1)

        self.classifier = nn.Sequential(
            nn.Linear(200*1*5, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200, 2),)
        
        
    def forward(self, x):
        print(x.shape)
        x = F.relu(self.a(x))
        print(x.shape)
        x = F.relu(self.b(x))
        print(x.shape)
        x = self.c(x)
        print(x.shape)
        x = self.d(x)
        print(x.shape)
        x = self.e(x)
        print(x.shape)
        x = F.relu(self.f(x))
        print(x.shape)
        x = self.g(x)
        print(x.shape)
        x = self.h(x)
        print('h', x.shape)
        x = self.i(x)
        print('i', x.shape)
        x = F.relu(self.j(x))
        print('j', x.shape)
        x = self.k(x)
        print('k', x.shape)
        x = self.l(x)
        print(x.shape)
        x = self.m(x)
        print(x.shape)
        x = F.relu(self.n(x))
        print(x.shape)
        x = self.o(x)
        print(x.shape)
        #x = self.p(x)
        #print('p', x.shape)
        x = self.r(x)
        
        
        x = x.view(-1, 200*1*5)
        #print(x.shape)
        x = self.classifier(x)
        print(x.shape)

        return x
net = Net()

3. Define a Loss function and optimizer
Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#criterion = nn.BCELoss()
#optimizer = optim.Adam(net.parameters())

4. Train the network
This is when things start to get interesting. We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

In [9]:
for epoch in range(30):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    running_loss = 0.0
   
    #inputs = torch.from_numpy(normalized_input_train).float()
    inputs = train_input #x_train
    inputs = torch.unsqueeze(inputs,1) # make it float and insert a fake batch dimension
    labels = train_target #y_train

    # wrap them in Variable
    inputs, labels = Variable(inputs), Variable(labels)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
 
    optimizer.step()

    running_loss += loss.data[0]
    print("Running loss", running_loss)
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]


Epoch  0
torch.Size([316, 1, 28, 500])
torch.Size([316, 25, 28, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 163])
torch.Size([316, 25, 1, 163])
torch.Size([316, 50, 1, 154])
torch.Size([316, 50, 1, 154])
h torch.Size([316, 50, 1, 51])
i torch.Size([316, 50, 1, 51])
j torch.Size([316, 100, 1, 42])
k torch.Size([316, 100, 1, 42])
torch.Size([316, 100, 1, 14])
torch.Size([316, 100, 1, 14])
torch.Size([316, 200, 1, 5])
torch.Size([316, 200, 1, 5])
torch.Size([316, 2])
Running loss 0.744152307510376

Epoch  1
torch.Size([316, 1, 28, 500])
torch.Size([316, 25, 28, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 163])
torch.Size([316, 25, 1, 163])
torch.Size([316, 50, 1, 154])
torch.Size([316, 50, 1, 154])
h torch.Size([316, 50, 1, 51])
i torch.Size([316, 50, 1, 51])
j torch.Size([316, 100, 1, 42])
k torch.Size([316, 100, 1, 42])
torch.Size([316, 100, 1, 14])
torch.Size([316, 100, 1, 14])
torch.Size([316,

torch.Size([316, 25, 28, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 163])
torch.Size([316, 25, 1, 163])
torch.Size([316, 50, 1, 154])
torch.Size([316, 50, 1, 154])
h torch.Size([316, 50, 1, 51])
i torch.Size([316, 50, 1, 51])
j torch.Size([316, 100, 1, 42])
k torch.Size([316, 100, 1, 42])
torch.Size([316, 100, 1, 14])
torch.Size([316, 100, 1, 14])
torch.Size([316, 200, 1, 5])
torch.Size([316, 200, 1, 5])
torch.Size([316, 2])
Running loss 0.6893907785415649

Epoch  16
torch.Size([316, 1, 28, 500])
torch.Size([316, 25, 28, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 491])
torch.Size([316, 25, 1, 163])
torch.Size([316, 25, 1, 163])
torch.Size([316, 50, 1, 154])
torch.Size([316, 50, 1, 154])
h torch.Size([316, 50, 1, 51])
i torch.Size([316, 50, 1, 51])
j torch.Size([316, 100, 1, 42])
k torch.Size([316, 100, 1, 42])
torch.Size([316, 100, 1, 14])
torch.Size([316, 100, 1, 14])
torch.Size([316, 200, 1, 5])
torch.Size([316, 200, 1, 

5. Test the network on the test data
We have trained the network for 2 passes over the training dataset. But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.

# Test Dataset - Prediction 

In [10]:
correct = 0
total = 0

#prediction = torch.from_numpy(normalized_input_test).float() #With normalized input 
prediction = test_input #x_train
prediction_squeezed = torch.unsqueeze(prediction,1) # make it float and insert a fake batch dimension
labels = test_target #y_train

# wrap them in Variable
predictionVar, labelsVar = Variable(prediction_squeezed), Variable(labels)

# forward + backward + optimize
#The outputs are energies for the 10 classes. Higher the energy for a class, the more the network thinks
#that the image is of the particular class. So, let’s get the index of the highest energy:
outputsPred = net(predictionVar) 

_, predicted = torch.max(outputsPred.data, 1) #predicted contains the predicted labels 
total += labels.size(0)
correct += (predicted == labels).sum()

print('Accuracy of the network on the test singals: %d %%' % (100 * correct / total))

torch.Size([100, 1, 28, 500])
torch.Size([100, 25, 28, 491])
torch.Size([100, 25, 1, 491])
torch.Size([100, 25, 1, 491])
torch.Size([100, 25, 1, 163])
torch.Size([100, 25, 1, 163])
torch.Size([100, 50, 1, 154])
torch.Size([100, 50, 1, 154])
h torch.Size([100, 50, 1, 51])
i torch.Size([100, 50, 1, 51])
j torch.Size([100, 100, 1, 42])
k torch.Size([100, 100, 1, 42])
torch.Size([100, 100, 1, 14])
torch.Size([100, 100, 1, 14])
torch.Size([100, 200, 1, 5])
torch.Size([100, 200, 1, 5])
torch.Size([100, 2])
Accuracy of the network on the test singals: 56 %


In [11]:
predicted.shape

predicted


 1
 1
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 1
 0
 1
 0
 1
 0
 1
 1
 1
 0
 0
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 0
 0
 1
 0
 1
[torch.LongTensor of size 100]

In [12]:
labels

labels.shape

torch.Size([100])